In [ ]:
!pip install transformers==4.28.0
!pip install datasets
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install pydantic
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import classification_report,f1_score,confusion_matrix
from fastapi import FastAPI, Body, Request
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from transformers import TextClassificationPipeline
from pydantic import BaseModel
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.5 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl 

In [ ]:
from typing import List

app = FastAPI(title="Climate Change Models on FastAPI")
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)
# Climate Detection
climate_change_detection_tokenizer = None
climate_change_detection_model = None
climate_change_detection_arg = None
climate_change_detection_trainer = None

# Climate Sentiment
climate_change_sentiment_tokenizer = None
climate_change_sentiment_model = None
climate_change_sentiment_arg = None
climate_change_sentiment_trainer = None

# Climate Specificity
climate_change_specificity_tokenizer = None
climate_change_specificity_model = None
climate_change_specificity_arg = None
climate_change_specificity_trainer = None

# Climate Commitments and Actions
climate_change_commitments_actions_tokenizer = None
climate_change_commitments_actions_model = None
climate_change_commitments_actions_arg = None
climate_change_commitments_actions_trainer = None

# Climate Classification on TCFD Recommendations
climate_change_tcfd_tokenizer = None
climate_change_tcfd_model = None
climate_change_tcfd_arg = None
climate_change_tcfd_trainer = None

# Our model based on ClimateBert Climate Detection model

climate_change_our_tokenizer = None
climate_change_our_model = None
climate_change_our_arg = None
climate_change_our_trainer = None

# Define Request Types
class RequestBodyMultipleObjects(BaseModel):
    objects: List | None = None

#Loading Climate Detection model

@app.on_event("startup")
def load_detection_model():
  global climate_change_detection_tokenizer
  global climate_change_detection_model
  global climate_change_detection_arg
  global climate_change_detection_trainer
  climate_change_detection_tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-detector")
  climate_change_detection_model = AutoModelForSequenceClassification.from_pretrained("climatebert/distilroberta-base-climate-detector")
  climate_change_detection_arg = TrainingArguments(
    "label",
    learning_rate=5e-5,
    num_train_epochs=4,
    per_device_eval_batch_size=32,
    per_device_train_batch_size=32,
    seed=19
  )
  climate_change_detection_trainer = Trainer(
    model=climate_change_detection_model,
    args=climate_change_detection_arg,
    tokenizer=climate_change_detection_tokenizer,
)

#Loading Climate Sentiment model

@app.on_event("startup")
def load_sentiment_model():
  global climate_change_sentiment_tokenizer
  global climate_change_sentiment_model
  global climate_change_sentiment_arg
  global climate_change_sentiment_trainer
  climate_change_sentiment_tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-sentiment")
  climate_change_sentiment_model = AutoModelForSequenceClassification.from_pretrained("climatebert/distilroberta-base-climate-sentiment")
  climate_change_sentiment_arg = TrainingArguments(
    "label",
    learning_rate=5e-5,
    num_train_epochs=4,
    per_device_eval_batch_size=32,
    per_device_train_batch_size=32,
    seed=19
  )
  climate_change_sentiment_trainer = Trainer(
    model=climate_change_sentiment_model,
    args=climate_change_sentiment_arg,
    tokenizer=climate_change_sentiment_tokenizer,
)

#Loading Climate Specificity model

@app.on_event("startup")
def load_specificity_model():
  global climate_change_specificity_tokenizer
  global climate_change_specificity_model
  global climate_change_specificity_arg
  global climate_change_specificity_trainer
  climate_change_specificity_tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-specificity")
  climate_change_specificity_model = AutoModelForSequenceClassification.from_pretrained("climatebert/distilroberta-base-climate-specificity")
  climate_change_specificity_arg = TrainingArguments(
    "label",
    learning_rate=5e-5,
    num_train_epochs=4,
    per_device_eval_batch_size=32,
    per_device_train_batch_size=32,
    seed=19
  )
  climate_change_specificity_trainer = Trainer(
    model=climate_change_specificity_model,
    args=climate_change_specificity_arg,
    tokenizer=climate_change_specificity_tokenizer,
)

#Loading Climate Commitments and Actions model

@app.on_event("startup")
def load_commitments_model():
  global climate_change_commitments_actions_tokenizer
  global climate_change_commitments_actions_model
  global climate_change_commitments_actions_arg
  global climate_change_commitments_actions_trainer
  climate_change_commitments_actions_tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-commitment")
  climate_change_commitments_actions_model = AutoModelForSequenceClassification.from_pretrained("climatebert/distilroberta-base-climate-commitment")
  climate_change_commitments_actions_arg = TrainingArguments(
    "label",
    learning_rate=5e-5,
    num_train_epochs=4,
    per_device_eval_batch_size=32,
    per_device_train_batch_size=32,
    seed=19
  )
  climate_change_commitments_actions_trainer = Trainer(
    model=climate_change_commitments_actions_model,
    args=climate_change_commitments_actions_arg,
    tokenizer=climate_change_commitments_actions_tokenizer,
)

#Loading Climate TCFD Recommendations model

@app.on_event("startup")
def load_tcfd_model():
  global climate_change_tcfd_tokenizer
  global climate_change_tcfd_model
  global climate_change_tcfd_arg
  global climate_change_tcfd_trainer
  climate_change_tcfd_tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-tcfd")
  climate_change_tcfd_model = AutoModelForSequenceClassification.from_pretrained("climatebert/distilroberta-base-climate-tcfd")
  climate_change_tcfd_arg = TrainingArguments(
    "label",
    learning_rate=5e-5,
    num_train_epochs=4,
    per_device_eval_batch_size=32,
    per_device_train_batch_size=32,
    seed=19
  )
  climate_change_tcfd_trainer = Trainer(
    model=climate_change_tcfd_model,
    args=climate_change_tcfd_arg,
    tokenizer=climate_change_tcfd_tokenizer,
)

#Loading our model for climate change

@app.on_event("startup")
def load_our_model():
  global climate_change_our_tokenizer
  global climate_change_our_model
  global climate_change_our_arg
  global climate_change_our_trainer
  climate_change_our_tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/DS-Environment-Project/Models/climate-change-model-based-on-climate-detector-further-trained-on-paragraphs")
  climate_change_our_model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/DS-Environment-Project/Models/climate-change-model-based-on-climate-detector-further-trained-on-paragraphs")
  climate_change_our_model.cuda()
  climate_change_our_model.to("cuda")
  climate_change_our_arg = TrainingArguments(
    "label",
    learning_rate=5e-5,
    num_train_epochs=4,
    per_device_eval_batch_size=32,
    per_device_train_batch_size=32,
    seed=19
  )
  climate_change_our_trainer = Trainer(
    model=climate_change_our_model,
    args=climate_change_our_arg,
    tokenizer=climate_change_our_tokenizer,
)








<ipython-input-2-3ced73633c4a>:54: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
<ipython-input-2-3ced73633c4a>:78: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
<ipython-input-2-3ced73633c4a>:102: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
<ipython-input-2-3ced73633c4a>:126: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Rea

In [ ]:
#Climate Detection
@app.post("/climate-detect")
def get_label_detect(input_data, chosen_model):
  dataset=load_dataset("climatebert/climate_detection")
  if chosen_model == "auto-detect":
    sentences = input_data.split(".")
    if len(sentences) > 1:
      pipe = TextClassificationPipeline(model=climate_change_detection_model, tokenizer=climate_change_detection_tokenizer, device="cuda:0")
    else:
      pipe = TextClassificationPipeline(model=climate_change_our_model, tokenizer=climate_change_our_tokenizer, device="cuda:0")
  elif chosen_model == "our-model":
    pipe = TextClassificationPipeline(model=climate_change_our_model, tokenizer=climate_change_our_tokenizer, device="cuda:0")
  else:
    pipe = TextClassificationPipeline(model=climate_change_detection_model, tokenizer=climate_change_detection_tokenizer, device="cuda:0")
  # encoded_input = climate_change_tokenizer(dataset['test']['text'],  truncation=True)
  predicted_data = pipe(input_data)
  return predicted_data[0]

In [ ]:
#Climate Sentiment
@app.post("/climate-sentiment")
def get_label_sentiment(input_data):
  dataset=load_dataset("climatebert/climate_sentiment")
  pipe = TextClassificationPipeline(model=climate_change_sentiment_model, tokenizer=climate_change_sentiment_tokenizer, device="cuda:0")
  # encoded_input = climate_change_tokenizer(dataset['test']['text'],  truncation=True)
  predicted_data = pipe(input_data)
  return predicted_data[0]

In [ ]:
#Climate Specificity
@app.post("/climate-specificity")
def get_label_specificity(input_data):
  dataset=load_dataset("climatebert/climate_specificity")
  pipe = TextClassificationPipeline(model=climate_change_specificity_model, tokenizer=climate_change_specificity_tokenizer, device="cuda:0")
  # encoded_input = climate_change_tokenizer(dataset['test']['text'],  truncation=True)
  predicted_data = pipe(input_data)
  return predicted_data[0]

In [ ]:
#Climate Commitments and Actions
@app.post("/climate-commitments-actions")
def get_label_commitments_actions(input_data):
  dataset=load_dataset("climatebert/climate_commitments_actions")
  pipe = TextClassificationPipeline(model=climate_change_commitments_actions_model, tokenizer=climate_change_commitments_actions_tokenizer, device="cuda:0")
  # encoded_input = climate_change_tokenizer(dataset['test']['text'],  truncation=True)
  predicted_data = pipe(input_data)
  return predicted_data[0]

In [ ]:
#Climate Classification on TCFD Recommendations
@app.post("/climate-tcfd")
def get_label_tcfd(input_data):
  dataset=load_dataset("climatebert/tcfd_recommendations")
  pipe = TextClassificationPipeline(model=climate_change_tcfd_model, tokenizer=climate_change_tcfd_tokenizer, device="cuda:0")
  # encoded_input = climate_change_tokenizer(dataset['test']['text'],  truncation=True)
  predicted_data = pipe(input_data)
  return predicted_data[0]

In [ ]:
#Training on Climate Detection task - ClimateBert model
@app.post("/train/climate-detect")
async def train_detection(request: Request):
  dataset=load_dataset("climatebert/climate_detection")
  json_object = await request.json()
  entries = json_object['entries']
  df = pd.DataFrame(data=eval(entries),columns=["text","label"])
  df["label"] = df["label"].replace("yes",1)
  df["label"] = df["label"].replace("no",0)
  train_ds = Dataset.from_pandas(df)
  train_encoded_ds = train_ds.map(lambda t: climate_change_detection_tokenizer(t['text'],  truncation=True, padding=True), batched=True,load_from_cache_file=False)
  climate_change_detection_trainer = Trainer(
    model=climate_change_detection_model,
    args=climate_change_detection_arg,
    tokenizer=climate_change_detection_tokenizer,
    train_dataset=train_encoded_ds
)
  climate_change_detection_trainer.train()
  return entries

In [ ]:
#Training on Climate Detection task - Our model
@app.post("/train/climate-detect-our")
async def train_detection_our(request: Request):
  dataset=load_dataset("climatebert/climate_detection")
  json_object = await request.json()
  entries = json_object['entries']
  df = pd.DataFrame(data=eval(entries),columns=["text","label"])
  df["label"] = df["label"].replace("yes",1)
  df["label"] = df["label"].replace("no",0)
  train_ds = Dataset.from_pandas(df)
  train_encoded_ds = train_ds.map(lambda t: climate_change_our_tokenizer(t['text'],  truncation=True, padding=True), batched=True,load_from_cache_file=False)
  climate_change_our_trainer = Trainer(
    model=climate_change_our_model,
    args=climate_change_our_arg,
    tokenizer=climate_change_our_tokenizer,
    train_dataset=train_encoded_ds
)
  climate_change_our_trainer.train()
  return entries

In [ ]:
#Training on Climate Sentiment task
@app.post("/train/climate-sentiment")
async def train_sentiment(request: Request):
  dataset=load_dataset("climatebert/climate_sentiment")
  json_object = await request.json()
  entries = json_object['entries']
  df = pd.DataFrame(data=eval(entries),columns=["text","label"])
  df["label"] = df["label"].replace("risk",0)
  df["label"] = df["label"].replace("neutral",1)
  df["label"] = df["label"].replace("opportunity",2)
  train_ds = Dataset.from_pandas(df)
  train_encoded_ds = train_ds.map(lambda t: climate_change_sentiment_tokenizer(t['text'],  truncation=True, padding=True), batched=True,load_from_cache_file=False)
  climate_change_sentiment_trainer = Trainer(
    model=climate_change_sentiment_model,
    args=climate_change_sentiment_arg,
    tokenizer=climate_change_sentiment_tokenizer,
    train_dataset=train_encoded_ds
)
  climate_change_sentiment_trainer.train()
  return entries

In [ ]:
#Training on Climate Specificity task
@app.post("/train/climate-specificity")
async def train_specificity(request: Request):
  dataset=load_dataset("climatebert/climate_specificity")
  json_object = await request.json()
  entries = json_object['entries']
  df = pd.DataFrame(data=eval(entries),columns=["text","label"])
  df["label"] = df["label"].replace("non-specific",0)
  df["label"] = df["label"].replace("specific",1)
  train_ds = Dataset.from_pandas(df)
  train_encoded_ds = train_ds.map(lambda t: climate_change_specificity_tokenizer(t['text'],  truncation=True, padding=True), batched=True,load_from_cache_file=False)
  climate_change_specificity_trainer = Trainer(
    model=climate_change_specificity_model,
    args=climate_change_specificity_arg,
    tokenizer=climate_change_specificity_tokenizer,
    train_dataset=train_encoded_ds
)
  climate_change_specificity_trainer.train()
  return entries

In [ ]:
#Training on Climate TCFD task
@app.post("/train/climate-tcfd")
async def train_tcfd(request: Request):
  dataset=load_dataset("climatebert/tcfd_recommendations")
  json_object = await request.json()
  entries = json_object['entries']
  df = pd.DataFrame(data=eval(entries),columns=["text","label"])
  df["label"] = df["label"].replace("none",0)
  df["label"] = df["label"].replace("metrics",1)
  df["label"] = df["label"].replace("strategy",2)
  df["label"] = df["label"].replace("risk",3)
  df["label"] = df["label"].replace("governance",4)
  train_ds = Dataset.from_pandas(df)
  train_encoded_ds = train_ds.map(lambda t: climate_change_tcfd_tokenizer(t['text'],  truncation=True, padding=True), batched=True,load_from_cache_file=False)
  climate_change_tcfd_trainer = Trainer(
    model=climate_change_tcfd_model,
    args=climate_change_tcfd_arg,
    tokenizer=climate_change_tcfd_tokenizer,
    train_dataset=train_encoded_ds
)
  climate_change_tcfd_trainer.train()
  return entries

In [ ]:
#Training on Climate Commitments Actions task
@app.post("/train/climate-commitments-actions")
async def train_commitments(request: Request):
  dataset=load_dataset("climatebert/climate_commitments_actions")
  json_object = await request.json()
  entries = json_object['entries']
  df = pd.DataFrame(data=eval(entries),columns=["text","label"])
  df["label"] = df["label"].replace("no",0)
  df["label"] = df["label"].replace("yes",1)
  train_ds = Dataset.from_pandas(df)
  train_encoded_ds = train_ds.map(lambda t: climate_change_commitments_actions_tokenizer(t['text'],  truncation=True, padding=True), batched=True,load_from_cache_file=False)
  climate_change_commitments_actions_trainer = Trainer(
    model=climate_change_commitments_actions_model,
    args=climate_change_commitments_actions_arg,
    tokenizer=climate_change_commitments_actions_tokenizer,
    train_dataset=train_encoded_ds
)
  climate_change_commitments_actions_trainer.train()
  return entries

In [ ]:
#Predicting whole dataset on Climate Detection task - ClimateBert model
@app.post("/dataset/climate-detect")
async def dataset_detection(request: Request):
  dataset=load_dataset("climatebert/climate_detection")
  json_object = await request.json()
  entries = json_object['entries']
  df = pd.DataFrame(data=eval(entries),columns=["text"])
  df = df.applymap(str)
  # df["label"] = df["label"].replace("yes",1)
  # df["label"] = df["label"].replace("no",0)
  test_ds = Dataset.from_pandas(df)
  test_encoded_ds = test_ds.map(lambda t: climate_change_detection_tokenizer(t['text'],  truncation=True, padding=True), batched=True,load_from_cache_file=False)
  y_pred_climate = climate_change_detection_trainer.predict(test_encoded_ds)
  y_pred_climate = y_pred_climate.predictions
  import numpy as np
  y_pred_climate = [np.argmax(y_pred_climate[i]) for i in range(0,len(y_pred_climate))]
  predictions = []
  for i in range(0, len(y_pred_climate)):
    if y_pred_climate[i] == 0:
      predictions.append("no")
    elif y_pred_climate[i] == 1:
      predictions.append("yes")
  return predictions

In [ ]:
#Predicting whole dataset on Climate sentiment task - ClimateBert model
@app.post("/dataset/climate-sentiment")
async def dataset_sentiment(request: Request):
  dataset=load_dataset("climatebert/climate_sentiment")
  json_object = await request.json()
  entries = json_object['entries']
  df = pd.DataFrame(data=eval(entries),columns=["text"])
  df = df.applymap(str)
  # df["label"] = df["label"].replace("yes",1)
  # df["label"] = df["label"].replace("no",0)
  test_ds = Dataset.from_pandas(df)
  test_encoded_ds = test_ds.map(lambda t: climate_change_sentiment_tokenizer(t['text'],  truncation=True, padding=True), batched=True,load_from_cache_file=False)
  y_pred_climate = climate_change_sentiment_trainer.predict(test_encoded_ds)
  y_pred_climate = y_pred_climate.predictions
  import numpy as np
  y_pred_climate = [np.argmax(y_pred_climate[i]) for i in range(0,len(y_pred_climate))]
  predictions = []
  for i in range(0, len(y_pred_climate)):
    if y_pred_climate[i] == 0:
      predictions.append("risk")
    elif y_pred_climate[i] == 1:
      predictions.append("neutral")
    elif y_pred_climate[i] == 2:
      predictions.append("opportunity")
  return predictions

In [ ]:
#Predicting whole dataset on Climate specificity task - ClimateBert model
@app.post("/dataset/climate-specificity")
async def dataset_specificity(request: Request):
  dataset=load_dataset("climatebert/climate_specificity")
  json_object = await request.json()
  entries = json_object['entries']
  df = pd.DataFrame(data=eval(entries),columns=["text"])
  df = df.applymap(str)
  # df["label"] = df["label"].replace("yes",1)
  # df["label"] = df["label"].replace("no",0)
  test_ds = Dataset.from_pandas(df)
  test_encoded_ds = test_ds.map(lambda t: climate_change_specificity_tokenizer(t['text'],  truncation=True, padding=True), batched=True,load_from_cache_file=False)
  y_pred_climate = climate_change_specificity_trainer.predict(test_encoded_ds)
  y_pred_climate = y_pred_climate.predictions
  import numpy as np
  y_pred_climate = [np.argmax(y_pred_climate[i]) for i in range(0,len(y_pred_climate))]
  predictions = []
  for i in range(0, len(y_pred_climate)):
    if y_pred_climate[i] == 0:
      predictions.append("non-specific")
    elif y_pred_climate[i] == 1:
      predictions.append("specific")
  return predictions

In [ ]:
#Predicting whole dataset on Climate commitments task - ClimateBert model
@app.post("/dataset/climate-commitments")
async def dataset_specificity(request: Request):
  dataset=load_dataset("climatebert/climate_commitments_actions")
  json_object = await request.json()
  entries = json_object['entries']
  df = pd.DataFrame(data=eval(entries),columns=["text"])
  df = df.applymap(str)
  # df["label"] = df["label"].replace("yes",1)
  # df["label"] = df["label"].replace("no",0)
  test_ds = Dataset.from_pandas(df)
  test_encoded_ds = test_ds.map(lambda t: climate_change_commitments_actions_tokenizer(t['text'],  truncation=True, padding=True), batched=True,load_from_cache_file=False)
  y_pred_climate = climate_change_commitments_actions_trainer.predict(test_encoded_ds)
  y_pred_climate = y_pred_climate.predictions
  import numpy as np
  y_pred_climate = [np.argmax(y_pred_climate[i]) for i in range(0,len(y_pred_climate))]
  predictions = []
  for i in range(0, len(y_pred_climate)):
    if y_pred_climate[i] == 0:
      predictions.append("no")
    elif y_pred_climate[i] == 1:
      predictions.append("yes")
  return predictions

In [ ]:
#Predicting whole dataset on Climate commitments task - ClimateBert model
@app.post("/dataset/climate-tcfd")
async def dataset_specificity(request: Request):
  dataset=load_dataset("climatebert/tcfd_recommendations")
  json_object = await request.json()
  entries = json_object['entries']
  df = pd.DataFrame(data=eval(entries),columns=["text"])
  df = df.applymap(str)
  # df["label"] = df["label"].replace("yes",1)
  # df["label"] = df["label"].replace("no",0)
  test_ds = Dataset.from_pandas(df)
  test_encoded_ds = test_ds.map(lambda t: climate_change_tcfd_tokenizer(t['text'],  truncation=True, padding=True), batched=True,load_from_cache_file=False)
  y_pred_climate = climate_change_tcfd_trainer.predict(test_encoded_ds)
  y_pred_climate = y_pred_climate.predictions
  import numpy as np
  y_pred_climate = [np.argmax(y_pred_climate[i]) for i in range(0,len(y_pred_climate))]
  predictions = []
  for i in range(0, len(y_pred_climate)):
    if y_pred_climate[i] == 0:
      predictions.append("none")
    elif y_pred_climate[i] == 1:
      predictions.append("metrics")
    elif y_pred_climate[i] == 2:
      predictions.append("strategy")
    elif y_pred_climate[i] == 3:
      predictions.append("risk")
    elif y_pred_climate[i] == 4:
      predictions.append("governance")
  return predictions

In [ ]:
#Evaluating on Climate Detection task - ClimateBert model
@app.post("/evaluate/climate-detect")
async def evaluate_detection(request: Request):
  dataset=load_dataset("climatebert/climate_detection")
  encoded_ds = dataset.map(lambda t: climate_change_detection_tokenizer(t['text'],  truncation=True, padding=True), batched=True,load_from_cache_file=False)
  y_pred = climate_change_detection_trainer.predict(encoded_ds['test'])
  y_pred = y_pred.predictions
  import numpy as np
  y_pred = [np.argmax(y_pred[i]) for i in range(0,len(y_pred))]
  from sklearn.metrics import f1_score
  f1 = f1_score(encoded_ds['test']['label'],y_pred,average='macro')
  return f1

In [ ]:
#Evaluating on Climate commitments_actions task - Our model
@app.post("/evaluate/climate-commitments-actions")
async def evaluate_detection(request: Request):
  dataset=load_dataset("climatebert/climate_commitments_actions")
  encoded_ds = dataset.map(lambda t: climate_change_commitments_actions_tokenizer(t['text'],  truncation=True, padding=True), batched=True,load_from_cache_file=False)
  y_pred = climate_change_commitments_actions_trainer.predict(encoded_ds['test'])
  y_pred = y_pred.predictions
  import numpy as np
  y_pred = [np.argmax(y_pred[i]) for i in range(0,len(y_pred))]
  from sklearn.metrics import f1_score
  f1 = f1_score(encoded_ds['test']['label'],y_pred,average='macro')
  return f1

In [ ]:
#Evaluating on Climate tcfd task - Our model
@app.post("/evaluate/climate-tcfd")
async def evaluate_detection(request: Request):
  dataset=load_dataset("climatebert/tcfd_recommendations")
  encoded_ds = dataset.map(lambda t: climate_change_tcfd_tokenizer(t['text'],  truncation=True, padding=True), batched=True,load_from_cache_file=False)
  y_pred = climate_change_tcfd_trainer.predict(encoded_ds['test'])
  y_pred = y_pred.predictions
  import numpy as np
  y_pred = [np.argmax(y_pred[i]) for i in range(0,len(y_pred))]
  from sklearn.metrics import f1_score
  f1 = f1_score(encoded_ds['test']['label'],y_pred,average='macro')
  return f1

In [ ]:
#Evaluating on Climate Detection task - Our model
@app.post("/evaluate/climate-detect-our")
async def evaluate_detection(request: Request):
  dataset=load_dataset("climatebert/climate_detection")
  encoded_ds = dataset.map(lambda t: climate_change_our_tokenizer(t['text'],  truncation=True, padding=True), batched=True,load_from_cache_file=False)
  y_pred = climate_change_our_trainer.predict(encoded_ds['test'])
  y_pred = y_pred.predictions
  import numpy as np
  y_pred = [np.argmax(y_pred[i]) for i in range(0,len(y_pred))]
  from sklearn.metrics import f1_score
  f1 = f1_score(encoded_ds['test']['label'],y_pred,average='macro')
  return f1

In [ ]:
#Evaluating on Climate Sentiment task
@app.post("/evaluate/climate-sentiment")
async def evaluate_detection(request: Request):
  dataset=load_dataset("climatebert/climate_sentiment")
  encoded_ds = dataset.map(lambda t: climate_change_sentiment_tokenizer(t['text'],  truncation=True, padding=True), batched=True,load_from_cache_file=False)
  y_pred = climate_change_sentiment_trainer.predict(encoded_ds['test'])
  y_pred = y_pred.predictions
  import numpy as np
  y_pred = [np.argmax(y_pred[i]) for i in range(0,len(y_pred))]
  from sklearn.metrics import f1_score
  f1 = f1_score(encoded_ds['test']['label'],y_pred,average='macro')
  return f1

In [ ]:
#Evaluating on Climate Specificity task
@app.post("/evaluate/climate-specificity")
async def evaluate_detection(request: Request):
  dataset=load_dataset("climatebert/climate_specificity")
  encoded_ds = dataset.map(lambda t: climate_change_specificity_tokenizer(t['text'],  truncation=True, padding=True), batched=True,load_from_cache_file=False)
  y_pred = climate_change_specificity_trainer.predict(encoded_ds['test'])
  y_pred = y_pred.predictions
  import numpy as np
  y_pred = [np.argmax(y_pred[i]) for i in range(0,len(y_pred))]
  from sklearn.metrics import f1_score
  f1 = f1_score(encoded_ds['test']['label'],y_pred,average='macro')
  return f1

In [ ]:
with open('/content/drive/MyDrive/ClimateCognize/ngrok-auth-token.txt', 'r') as file:
    token = file.readline()

ngrok.set_auth_token(token)

In [ ]:
ngrok_tunnel = ngrok.connect(8000,domain="social-glowworm-infinitely.ngrok-free.app")
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [221]
INFO:     Waiting for application startup.


Public URL: https://social-glowworm-infinitely.ngrok-free.app


(…)ector/resolve/main/tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

(…)climate-detector/resolve/main/vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

(…)climate-detector/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Step,Training Loss


INFO:     92.53.48.227:0 - "POST /train/climate-sentiment HTTP/1.1" 200 OK


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/320 [00:00<?, ? examples/s]

INFO:     92.53.48.227:0 - "POST /evaluate/climate-sentiment HTTP/1.1" 200 OK
